In [1]:
# Basic libraries
import time
import argparse

from os.path import abspath
from os.path import dirname as up
import numpy as np
import sys
import h5py
import math

# Insert path to pybf library to system path
print(up(up(up(abspath("__file__")))))
path_to_lib = up(up(up(abspath("__file__"))))
print(path_to_lib)
sys.path.insert(0, path_to_lib)

#Import libraries and functions
from pybf.pybf.io_interfaces import DataLoader
from pybf.pybf.signal_processing import demodulate_decimate
from pybf.pybf.signal_processing import interpolate_modulate
from pybf.pybf.signal_processing import filter_band_pass
from pybf.scripts.beamformer_cartesian_realtime import BFCartesianRealTime
from pybf.pybf.transducer import Transducer
from pybf.pybf.image_settings import ImageSettings
from pybf.pybf.visualization import plot_image
from pybf.scripts.beamformer_cartesian import beamformer_cartesian

/home/sem21f11/Desktop
/home/sem21f11/Desktop


/home/sem21f11/miniconda3/envs/pybf_env/lib/python3.6/importlib/__init__.py:126: MatplotlibDeprecationWarning: 
The matplotlib.backends.backend_qt4agg backend was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  return _bootstrap._gcd_import(name[level:], package, level)


In [2]:
def get_original_rf_data(path_to_dataset):
    
    # Read file
    data_loader_obj = DataLoader(path_to_dataset)
    
    rf_data_shape = (data_loader_obj.num_of_acq_per_frame,) + data_loader_obj.get_rf_data(0, 0).shape
    print('RF data shape: ', rf_data_shape)

    rf_data = np.zeros(rf_data_shape)
    for i in range(rf_data.shape[0]):
        rf_data[i, :, :] = data_loader_obj.get_rf_data(0, i)
        
    return rf_data

In [3]:
rf_original_data = get_original_rf_data("../datasets/archive/archive_to_download/database/experiments/contrast_speckle/rf_dataset.hdf5")

print(rf_original_data[0][1][223].dtype)

RF data shape:  (75, 128, 3328)
float64


In [4]:
# Load integer conversion output values from file
import numpy as np
with open('./BL_Coding/resolution_distorsion/int_converted_values.npy', 'rb') as f:
    int_out_values = np.load(f)
f.close()
print(int_out_values.shape)
#print(int_converted_data[2][2][3])
print(int_out_values[2][2][3])

(75, 128, 3328)
2048


In [5]:
!pip install pvlib
!pip install ipympl
!pip install progressbar
import ipympl
import pvlib as pvl
%matplotlib widget

In [6]:
# Contrast speckle distribution

import matplotlib.pyplot as plt
%matplotlib widget

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7,6), sharex=True)
print(int_out_values[0][0][:].shape)

spec_bins = [0, 500, 1000, 1500, 1800, 2000, 2020, 2040, 2060, 2080, 2100, 2150, 2200, 2300, 2400, 2500, 3000, 3500, 4000, 4100]
plt.hist(int_out_values.flatten(), bins = spec_bins)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3328,)


In [5]:
# Load two arrays

# Load integer conversion output values from file

# Contrast_speckle
import numpy as np
with open('./BL_Coding/resolution_distorsion/int_converted_values.npy', 'rb') as f:
    contrast_speckle = np.load(f)
f.close()
print(contrast_speckle.shape)
print(contrast_speckle[2][2][3])
print(np.amax(contrast_speckle))
print(np.amin(contrast_speckle))

# Contrast_speckle
import numpy as np
with open('./BL_Coding/resolution_distorsion/int_converted_values.npy', 'rb') as f:
    resolution_distorsion = np.load(f)
f.close()
print(resolution_distorsion.shape)
print(resolution_distorsion[2][2][3])
print(np.amax(resolution_distorsion))
print(np.amin(resolution_distorsion))



(75, 128, 3328)
2048
4095
48
(75, 128, 3328)
2048
4095
48


In [6]:
# Function to perform requantization and inverse requantization
# Name input_array either resolution_distorsion or contrast_speckle
def requantization_and_inverse(bits, input_string, input_array, old_min_1, old_max_1, new_min_1, new_max_1, old_min_2, old_max_2, new_min_2, new_max_2, old_min_3, old_max_3, new_min_3, new_max_3):

    # Requantize data in x bits
    print(input_array.shape)

    # Ranges
    newmax_1 = new_max_1
    newmin_1 = new_min_1
    newran_1 = newmax_1 - newmin_1

    oldmax_1 = old_max_1
    oldmin_1 = old_min_1
    oldran_1 = oldmax_1 - oldmin_1

    newmax_2 = new_max_2
    newmin_2 = new_min_2
    newran_2 = newmax_2 - newmin_2

    oldmax_2 = old_max_2
    oldmin_2 = old_min_2
    oldran_2 = oldmax_2 - oldmin_2

    newmax_3 = new_max_3
    newmin_3 = new_min_3
    newran_3 = newmax_3 - newmin_3

    oldmax_3 = old_max_3
    oldmin_3 = old_min_3
    oldran_3 = oldmax_3 - oldmin_3


    # Requantize data
    requant = np.zeros((input_array.shape), dtype=np.int32)
    flat_requant = requant.flatten()
    print(flat_requant.shape)
    index = 0
    for sample in input_array.flatten():
        if ((sample >= oldmin_1) and (sample < oldmax_1)):
            flat_requant[index] = int(np.round((((sample-oldmin_1)*newran_1)/oldran_1) + newmin_1))

        elif ((sample >= oldmin_2) and (sample < oldmax_2)):
            flat_requant[index] = int(np.round((((sample-oldmin_2)*newran_2)/oldran_2) + newmin_2))

        elif ((sample >= oldmin_3) and (sample <= oldmax_3)):
            flat_requant[index] = int(np.round((((sample-oldmin_3)*newran_3)/oldran_3) + newmin_3))    

        index = index + 1
        sys.stdout.write('\r')
        sys.stdout.write("{} out of 31948800".format(index))

    print(flat_requant.shape)

    requantized_data = flat_requant.reshape(input_array.shape)

    # Write the integer output values to a file
    with open('./Requantization/' + input_string + '_forw_requant_' + str(bits) + '_data.npy', 'wb') as f:
        np.save(f, requantized_data)

    f.close()

    # Calculate compression ratio

    # Initial size
    initial_number_bits = (input_array.size)*12
    print("Initial size: {} bits.".format(initial_number_bits))

    # Compressed number of bits
    # x bits per sample of requantized samples

    # Calculate number of samples kept
    final_number_bits = (requantized_data.size)*bits
    print("Final size: {} bits.".format(final_number_bits))

    # Compression ratio
    compr_ratio = (final_number_bits/initial_number_bits)*100
    print("The compressed data is {}% of the original data.".format(compr_ratio))

    # Convert back to 12-bits by performing the inverse requantization

    # Ranges
    newmax_1 = old_max_1
    newmin_1 = old_min_1
    newran_1 = newmax_1 - newmin_1

    oldmax_1 = new_max_1
    oldmin_1 = new_min_1
    oldran_1 = oldmax_1 - oldmin_1

    newmax_2 = old_max_2
    newmin_2 = old_min_2
    newran_2 = newmax_2 - newmin_2

    oldmax_2 = new_max_2
    oldmin_2 = new_min_2
    oldran_2 = oldmax_2 - oldmin_2

    newmax_3 = old_max_3
    newmin_3 = old_min_3
    newran_3 = newmax_3 - newmin_3

    oldmax_3 = new_max_3
    oldmin_3 = new_min_3
    oldran_3 = oldmax_3 - oldmin_3

    # Inversely Requantize data
    inv_requant = np.zeros((requantized_data.shape), dtype=np.int32)
    flat_inv_requant = inv_requant.flatten()
    print(flat_inv_requant.shape)
    index = 0
    for sample in requantized_data.flatten():
        if ((sample >= oldmin_1) and (sample < oldmax_1)):
            flat_inv_requant[index] = int(np.round((((sample-oldmin_1)*newran_1)/oldran_1) + newmin_1))

        elif ((sample >= oldmin_2) and (sample < oldmax_2)):
            flat_inv_requant[index] = int(np.round((((sample-oldmin_2)*newran_2)/oldran_2) + newmin_2))

        elif ((sample >= oldmin_3) and (sample <= oldmax_3)):
            flat_inv_requant[index] = int(np.round((((sample-oldmin_3)*newran_3)/oldran_3) + newmin_3))    

        index = index + 1
        sys.stdout.write('\r')
        sys.stdout.write("{} out of 31948800".format(index))


    print(flat_inv_requant.shape)

    inv_requantized_data = flat_inv_requant.reshape(requantized_data.shape)

    # Write the integer output values to a file
    with open('./Requantization/' + input_string + '_inv_requant_' + str(bits) + '_data.npy', 'wb') as f:
        np.save(f, inv_requantized_data)

    f.close()

    # Plot compressed image after requantization

    # Beamform and plot the compressed image

    dataset_path = "../datasets/archive/archive_to_download/database/experiments/" + input_string + "/rf_dataset.hdf5"

    data_loader_obj = DataLoader(dataset_path)

    ### Specify Image settings and create corresponding object ###

    img_res = [400, 600]
    image_x_range = [-0.019, 0.019]
    image_z_range = [0.005, 0.05]

    db_range = 50

    LATERAL_PIXEL_DENSITY_DEFAULT = 5

    img_config = ImageSettings(image_x_range[0],
                               image_x_range[1],
                               image_z_range[0],
                               image_z_range[1],
                               LATERAL_PIXEL_DENSITY_DEFAULT,
                               data_loader_obj.transducer)

    ### Specify preprocessing parameters for RF data ###

    decimation_factor = 1
    interpolation_factor = 10

    ### Specify TX strategy and Apodization parameters ###

    start_time = 0
    correction_time_shift = 0

    alpha_fov_apod = 40

    ### Specify Sampling Frequency ###

    SAMPLING_FREQ = 20.832 * (10 ** 6)

    filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

    bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                             data_loader_obj.tx_strategy,
                             data_loader_obj.transducer,
                             decimation_factor,
                             interpolation_factor,
                             img_res,
                             img_config,
                             start_time=start_time,
                             correction_time_shift=correction_time_shift,
                             alpha_fov_apod=alpha_fov_apod,
                             bp_filter_params=filters_params,
                             envelope_detector='hilbert',
                             picmus_dataset=True)


    # Beamforming
    img_data = bf.beamform(inv_requantized_data, numba_active=True)

    _ = plot_image(np.abs(img_data), 
                   scatters_coords_xz=None,
                   elements_coords_xz=None,
                   framework='plotly',
                   title='Requantization_' + str(bits) + '_bits_' + input_string,
                   image_x_range=image_x_range,
                   image_z_range=image_z_range,
                   db_range=db_range,
                   colorscale='Greys',
                   save_fig=False, 
                   show=True,
                   path_to_save='./Requantization')


    # Prepare dataset to extract metrics of contrast and speckle

    # Read info from file
    f_d = h5py.File("../datasets/archive/archive_to_download/database/experiments/" + input_string + "/" + input_string + "_expe_dataset_rf.hdf5", "r")

    attributes = f_d.attrs

    US = f_d['US']
    attributes = US.attrs

    US_DATASET0000 = US['US_DATASET0000']
    attributes = US_DATASET0000.attrs

    type_s = attributes.__getitem__('type')

    attributes = US_DATASET0000.attrs

    keys_1 = list(f_d.keys());

    US = f_d['US']

    US_DATASET0000 = US['US_DATASET0000']

    data = US_DATASET0000['data']

    imag_n = data['imag']

    real_n = data['real'][()]

    PRF = US_DATASET0000['PRF']

    angles = US_DATASET0000['angles']

    initial_time = US_DATASET0000['initial_time']

    modulation_frequency = US_DATASET0000['modulation_frequency']

    probe_geometry = US_DATASET0000['probe_geometry']

    sampling_frequency = US_DATASET0000['sampling_frequency']

    sound_speed = US_DATASET0000['sound_speed']

    # Create dataset suitable for calculating metrics

    imag_np = np.zeros((75, 128, 3328))

    # Creation of dataset

    out_filename = './Requantization/requant_' + str(bits) + '_' + input_string + '_compressed_dataset.hdf5'
    comp_file = h5py.File(out_filename,'w')
    comp_file.close()
    comp_file = h5py.File(out_filename,'a')

    # Complete the structure

    comp_file.attrs.create('version', [b'v.0.0.40'], dtype='|S9')

    US_path = 'US'
    US_group = comp_file.require_group(US_path)

    US_dataset_group = US_group.require_group('US_DATASET0000')

    US_dataset_group.attrs.create('type', [b"US"], dtype='|S2')
    US_dataset_group.attrs.create('subtype', [b"CPW"], dtype='|S3')
    US_dataset_group.attrs.create('signal_format', [b"RF"], dtype='|S2')
    US_dataset_group.attrs.create('name', [b"CIRS 040GSE Wires"], dtype='|S18')
    US_dataset_group.attrs.create('version', [b"v1.96"], dtype='|S6')
    US_dataset_group.attrs.create('creation_date', [b"2016/03/9 17:25:22.38"], dtype='|S22')

    # angles
    US_dataset_group.create_dataset( 'angles', data=angles)

    # PRF
    US_dataset_group.create_dataset( 'PRF', data=PRF)

    # initial_time
    US_dataset_group.create_dataset( 'initial_time', data=initial_time)

    # modulation frequency
    US_dataset_group.create_dataset( 'modulation_frequency', data=modulation_frequency)

    # probe geometry
    US_dataset_group.create_dataset( 'probe_geometry', data=probe_geometry)

    # sampling frequency
    US_dataset_group.create_dataset( 'sampling_frequency', data=sampling_frequency)

    # sound speed
    US_dataset_group.create_dataset( 'sound_speed', data=sound_speed)

    # imag data
    data_group = US_dataset_group.require_group('data')
    data_group.create_dataset('imag', data=imag_np)

    # real data
    divider = ((2**12)-1)/2
    rf_range_data = inv_requantized_data/divider
    rf_data = rf_range_data - 1
    data_group.create_dataset('real', data=rf_data)

    comp_file.close()


In [11]:
# Run function for quantization with different number of bits

# Contrast speckle

requantization_and_inverse(7, 'contrast_speckle', contrast_speckle, 0,2000,0,9,2000,2100,10,109,2100,4095,109,127)
requantization_and_inverse(8, 'contrast_speckle', contrast_speckle, 0,1950,0,24,1950,2150,25,224,2150,4095,225,255)
requantization_and_inverse(9, 'contrast_speckle', contrast_speckle, 0,1800,0,44,1800,2200,45,444,2200,4095,445,511)

# Resolution distorsion
requantization_and_inverse(7, 'resolution_distorsion', resolution_distorsion, 0,2000,0,9,2000,2100,10,109,2100,4095,109,127)
requantization_and_inverse(8, 'resolution_distorsion', resolution_distorsion, 0,1950,0,24,1950,2150,25,224,2150,4095,225,255)
requantization_and_inverse(9, 'resolution_distorsion', resolution_distorsion, 0,1800,0,44,1800,2200,45,444,2200,4095,445,511)

(75, 128, 3328)
(31948800,)
31948800 out of 31948800(31948800,)
Initial size: 383385600 bits.
Final size: 223641600 bits.
The compressed data is 58.333333333333336% of the original data.
(31948800,)
31948800 out of 31948800(31948800,)
The highest resolution for the system is:  (633, 205)
Delays precalculation...
TX strategy: plane waves
Number of plane waves:  75
Maximum angle:  16.0 °
Apodization precalculation...
Beamforming...
 
Time of execution: 45.09599208831787 seconds
BF Final dB range (-50.0,0.0)
(75, 128, 3328)
(31948800,)
31948800 out of 31948800(31948800,)
Initial size: 383385600 bits.
Final size: 255590400 bits.
The compressed data is 66.66666666666666% of the original data.
(31948800,)
31948800 out of 31948800(31948800,)
The highest resolution for the system is:  (633, 205)
Delays precalculation...
TX strategy: plane waves
Number of plane waves:  75
Maximum angle:  16.0 °
Apodization precalculation...
Beamforming...
 
Time of execution: 52.51710367202759 seconds
BF Final 

In [7]:
# Load integer conversion output values from file
import numpy as np
input_string = 'contrast_speckle'
bits = 8

with open('./Requantization/contrast_speckle_inv_requant_8_data.npy', 'rb') as f:
    inv_requant_values = np.load(f)
f.close()
print(inv_requant_values.shape)
#print(inv_requant_values[2][2][3])
print(inv_requant_values[2][2][3])

# Prepare dataset to extract metrics of contrast and speckle

# Read info from file
f_d = h5py.File("../datasets/archive/archive_to_download/database/experiments/" + input_string + "/" + input_string + "_expe_dataset_rf.hdf5", "r")

attributes = f_d.attrs

US = f_d['US']
attributes = US.attrs

US_DATASET0000 = US['US_DATASET0000']
attributes = US_DATASET0000.attrs

type_s = attributes.__getitem__('type')

attributes = US_DATASET0000.attrs

keys_1 = list(f_d.keys());

US = f_d['US']

US_DATASET0000 = US['US_DATASET0000']

data = US_DATASET0000['data']

imag_n = data['imag']

real_n = data['real'][()]

PRF = US_DATASET0000['PRF']

angles = US_DATASET0000['angles']

initial_time = US_DATASET0000['initial_time']

modulation_frequency = US_DATASET0000['modulation_frequency']

probe_geometry = US_DATASET0000['probe_geometry']

sampling_frequency = US_DATASET0000['sampling_frequency']

sound_speed = US_DATASET0000['sound_speed']

# Create dataset suitable for calculating metrics

imag_np = np.zeros((75, 128, 3328))

# Creation of dataset

out_filename = './Requantization/requant_' + str(bits) + '_' + input_string + '_compressed_dataset.hdf5'
comp_file = h5py.File(out_filename,'w')
comp_file.close()
comp_file = h5py.File(out_filename,'a')

# Complete the structure

comp_file.attrs.create('version', [b'v.0.0.40'], dtype='|S9')

US_path = 'US'
US_group = comp_file.require_group(US_path)

US_dataset_group = US_group.require_group('US_DATASET0000')

US_dataset_group.attrs.create('type', [b"US"], dtype='|S2')
US_dataset_group.attrs.create('subtype', [b"CPW"], dtype='|S3')
US_dataset_group.attrs.create('signal_format', [b"RF"], dtype='|S2')
US_dataset_group.attrs.create('name', [b"CIRS 040GSE Wires"], dtype='|S18')
US_dataset_group.attrs.create('version', [b"v1.96"], dtype='|S6')
US_dataset_group.attrs.create('creation_date', [b"2016/03/9 17:25:22.38"], dtype='|S22')

# angles
US_dataset_group.create_dataset( 'angles', data=angles)

# PRF
US_dataset_group.create_dataset( 'PRF', data=PRF)

# initial_time
US_dataset_group.create_dataset( 'initial_time', data=initial_time)

# modulation frequency
US_dataset_group.create_dataset( 'modulation_frequency', data=modulation_frequency)

# probe geometry
US_dataset_group.create_dataset( 'probe_geometry', data=probe_geometry)

# sampling frequency
US_dataset_group.create_dataset( 'sampling_frequency', data=sampling_frequency)

# sound speed
US_dataset_group.create_dataset( 'sound_speed', data=sound_speed)

# imag data
data_group = US_dataset_group.require_group('data')
data_group.create_dataset('imag', data=imag_np)

# real data
divider = ((2**12)-1)/2
rf_range_data = inv_requant_values/divider
rf_data = rf_range_data - 1
data_group.create_dataset('real', data=rf_data)

comp_file.close()

(75, 128, 3328)
2048
